# Object Detection: Furniture - Couches, Chairs, Tables and Beds
Below we will do the following:
1. Load images of couches, chairs, tables and beds and their corresponding bounding boxes.
2. Build an object detection model for identifying the location of couches, chairs, tables and beds in images.
3. Convert the model to CoreML and upload it to Skafos.

The example is based on [Turi Create's Object Detection model](https://github.com/apple/turicreate/tree/master/userguide/object_detection).

## Data Preparation and Model Training
The data for this example are 324 images: 100 chairs, 99 tables, 91 beds and 34 couches. The original data was taken from Google's open source [Open Images Dataset](https://storage.googleapis.com/openimages/web/index.html).

Once loaded, the data is randomly split into train and test sets, where 80% of the data is used for training and 20% is used for model evaluation (if you desire). Training this model with a GPU is much faster than CPU time. At this moment, Azure doesn't support GPU training on notebook instances.

In [ ]:
# Import libraries
import tarfile
import urllib
import turicreate as tc
import os

In [ ]:
# Specify the data set download url
data_url = "https://s3.amazonaws.com/skafos.example.data/ObjectDetection/couch_chair_table_bed.sframe.tar.gz"
data_path = "couch_chair_table_bed.sframe.tar.gz"

# Pull the compressed data and extract it
retrieve = urllib.request.urlretrieve(data_url, data_path)
tar = tarfile.open(data_path)
tar.extractall()
tar.close()

In [ ]:
## Load the data
data =  tc.SFrame('couch_chair_table_bed.sframe')

## Make a train-test split
train_data, test_data = data.random_split(0.8)

In [ ]:
train_data.head()

In [ ]:
# Create an object detection model
model = tc.object_detector.create(
    dataset=train_data,
    max_iterations=10 # to improve model accuracy, remove the max_iterations argument and TuriCreate will tune to an appropriate number
)

# Object Detector Training Docs:
# https://apple.github.io/turicreate/docs/api/generated/turicreate.object_detector.create.html#turicreate.object_detector.create

In [ ]:
model.evaluate(test_data)

## Model Export and Skafos Upload
- Convert the model to CoreML format so that it can run on an iOS device. Then deliver the model to your apps with **[Skafos](https://skafos.ai)**.

- If you don't already have an account, Sign Up for one **[here](https://dashboard.skafos.ai)**. 
- Once you've signed up for an account, grab an API token from your account settings.

In [ ]:
# Specify the CoreML model name
model_name = 'ObjectDetection'
coreml_model_name = model_name + '.mlmodel'

# Export the trained model to CoreML format
res = model.export_coreml(coreml_model_name)

In [ ]:
import skafos
from skafos import models
import os

# Set your API Token first for repeated use
os.environ["SKAFOS_API_TOKEN"] = "<YOUR-SKAFOS-API-TOKEN>"

# You can retrieve this info with skafos.summary()
org_name = "<YOUR-SKAFOS-ORG-NAME>"    # Example: "mike-gmail-com-467h2"
app_name = "<YOUR-SKAFOS-APP-NAME>"    # Example: "ObjectDetection-App"
model_name = "<YOUR-MODEL-NAME>"       # Example: "ObjectDetectionModel"

# Upload model version to Skafos
model_upload_result = models.upload_version(
    files="ObjectDetection.mlmodel",
    org_name=org_name,
    app_name=app_name,
    model_name=model_name
)